<a href="https://colab.research.google.com/github/gayathribaby05/Real-Time-Traffic-Monitoring-System/blob/main/traffic_monitoring.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install torch torchvision opencv-python pandas

In [3]:
import cv2
import numpy as np
import torch
import pandas as pd

# Constants
VIDEO_PATH = "/content/drive/MyDrive/Colab Notebooks/traffic_video.mp4"
RED_LINE_Y = 400  # Adjust based on resolution
OUTPUT_VIDEO = "output_annotated_video.mp4"
LOG_FILE = "traffic_logs.csv"

# Initialize YOLO model (using pretrained YOLOv5s)
model = torch.hub.load('ultralytics/yolov5', 'yolov5s')

# Initialize log storage
logs = []

# Function to annotate frame
def annotate_frame(frame, results, frame_number):
    global logs
    for *box, conf, cls in results.xyxy[0]:
        xmin, ymin, xmax, ymax = map(int, box)
        center_y = (ymin + ymax) / 2

        # Check if the vehicle crossed the red line
        crossed = center_y > RED_LINE_Y
        color = (0, 0, 255) if crossed else (0, 255, 0)
        status = "Crossed" if crossed else "Safe"

        # Draw bounding box and status
        cv2.rectangle(frame, (xmin, ymin), (xmax, ymax), color, 2)
        cv2.putText(frame, status, (xmin, ymin - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

        # Log the vehicle's status
        if crossed:
            logs.append({"Frame": frame_number, "Left": xmin, "Top": ymin,
                         "Right": xmax, "Bottom": ymax, "Status": status})

    return frame

# Process video
cap = cv2.VideoCapture(VIDEO_PATH)
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = None
frame_number = 0

# Check if video file is loaded
if not cap.isOpened():
    print(f"Error: Unable to open video file {VIDEO_PATH}")
    exit()

while True:
    ret, frame = cap.read()
    if not ret:
        print("End of video or unable to read frame.")
        break

    # Initialize video writer only once after reading the first valid frame
    if out is None:
        if frame is not None:
            out = cv2.VideoWriter(OUTPUT_VIDEO, fourcc, 60,
                                  (frame.shape[1], frame.shape[0]))
            print(f"Video writer initialized: {OUTPUT_VIDEO}")
        else:
            print("Error: Unable to access frame properties for initialization.")
            break

    # Perform detection
    results = model(frame)

    # Annotate frame
    annotated_frame = annotate_frame(frame, results, frame_number)

    # Draw the red line
    cv2.line(annotated_frame, (0, RED_LINE_Y), (frame.shape[1], RED_LINE_Y), (0, 0, 255), 2)

    # Write frame to output
    out.write(annotated_frame)
    frame_number += 1

# Release resources
cap.release()
if out:
    out.release()
    print(f"Annotated video saved: {OUTPUT_VIDEO}")
else:
    print("No output video generated. Please check the input video or processing logic.")

# Save logs to CSV
if logs:
    df = pd.DataFrame(logs)
    df.to_csv(LOG_FILE, index=False)
    print(f"Logs saved: {LOG_FILE}")
else:
    print("No logs generated. No vehicles crossed the red line.")


/usr/local/lib/python3.10/dist-packages/torch/hub.py:330: UserWarning: You are about to download and run code from an untrusted repository. In a future release, this won't be allowed. To add the repository to your trusted list, change the command to {calling_fn}(..., trust_repo=False) and a command prompt will appear asking for an explicit confirmation of trust, or load(..., trust_repo=True), which will assume that the prompt is to be answered with 'yes'. You can also use load(..., trust_repo='check') which will only prompt for confirmation if the repo is not already trusted. This will eventually be the default behaviour
  warnings.warn(
Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to /root/.cache/torch/hub/master.zip


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


YOLOv5 🚀 2024-12-5 Python-3.10.12 torch-2.5.1+cu121 CPU

100%|██████████| 14.1M/14.1M [00:00<00:00, 134MB/s]

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


Error: Unable to open video file /content/drive/MyDrive/Colab Notebooks/traffic_video.mp4
End of video or unable to read frame.
No output video generated. Please check the input video or processing logic.
No logs generated. No vehicles crossed the red line.
